In [ ]:
# imports

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import datetime
import seaborn as sns

sns.set_style("darkgrid")
%matplotlib inline

In [ ]:
# Dataset : https://www.kaggle.com/datasets/jeanmidev/smart-meters-in-london

In [ ]:
household_data = pd.read_csv("./dataset/informations_households.csv", encoding="utf-8")
print(f"There are {household_data['LCLid'].nunique()} households in information_households.csv")
print("Sample:")
household_data.head(3)

In [ ]:
# Breakdown of Tariff based on Acorn Group

affluency_df = household_data[household_data["Acorn_grouped"].isin(["Adversity", "Affluent", "Comfortable"])][["Acorn_grouped", "stdorToU", "LCLid"]]
affluency_count = affluency_df.groupby(["Acorn_grouped"]).count().reset_index()
affluency_grouped = affluency_df.groupby(["Acorn_grouped", "stdorToU"]).count().reset_index()
affluency_grouped["hh_count"] = affluency_grouped["Acorn_grouped"].apply(lambda a: affluency_grouped[affluency_grouped["Acorn_grouped"] == a]["LCLid"].sum())
affluency_grouped["hh_percent"] = affluency_grouped["LCLid"] / affluency_grouped["hh_count"] * 100

chart = sns.barplot(x="Acorn_grouped", y="hh_percent", hue="stdorToU", data=affluency_grouped)
chart.set_xlabel("Acorn Group")
chart.set_ylabel("Percent Households")
chart.set_title("Tariff Breakdown")
chart.legend(loc = "upper left", bbox_to_anchor=(1,1), title="Tariff");

In [ ]:
acorn_data = pd.read_csv("./dataset/acorn_details.csv", encoding="ISO-8859-1")

In [ ]:
# Get and clean data from block files, taken randomly file_count at a time

hh_block_path = Path("./dataset/halfhourly_dataset/halfhourly_dataset/")

def get_data_from_block_files(filepath, file_count = 10):
    # Get a random set of files and concatenate

    block_list = list(filepath.glob("*.csv"))
    file_list = random.choice(block_list, file_count, replace=False)
    final_df = pd.DataFrame()
    for file in file_list:
        # read file and filter only 2013 data
        file_name = str(file).split("\\")[-1]
        print(f"getting data from file: {file_name}")
        df = pd.read_csv(file, parse_dates=["tstp"])
        year_2013_mask = (df["tstp"] >= "2013-1-1") & (df["tstp"] <= "2013-12-31")
        final_df = pd.concat([final_df, df[year_2013_mask]])
        final_df.dropna(how="any", inplace=True)


    return final_df.reset_index(drop=True)

block_data = get_data_from_block_files(hh_block_path)
        

In [ ]:
# clean up 
block_data.rename(columns={"tstp":"datetime", "energy(kWh/hh)":"kWh"}, inplace=True)
block_data.drop(block_data[block_data["kWh"].isin(["Null"])].index, inplace=True)

print(f"Dataset has {block_data.shape[0]} rows from {block_data.drop_duplicates(subset=['LCLid']).shape[0]} households.")

In [ ]:
# Datetime breakdown

block_data["kWh"] = pd.to_numeric(block_data["kWh"])
block_data["quarter"] = block_data["datetime"].dt.quarter
block_data["date"] = block_data["datetime"].dt.date
block_data["month"] = block_data["datetime"].dt.month
block_data["time"] = block_data["datetime"].apply(lambda a: a.strftime("%I:%M %p"))

# pivot the data
# df_pivoted = block_data.pivot_table(index = ["LCLid", "date"], columns=["time"], values=["kWh"], sort=False).reset_index()

In [ ]:
# merge the household, acorn and power data

merged_df = block_data.merge(household_data[["LCLid", "Acorn", "Acorn_grouped", "stdorToU"]], how="left", left_on="LCLid", right_on="LCLid", copy=False)
acorn_group_mask = merged_df["Acorn_grouped"].isin(["Affluent", "Adversity", "Comfortable"])
merged_df = merged_df[acorn_group_mask].reset_index(drop=True)

In [ ]:
# Assign rates

peak_rate = 67.20
off_peak_rate = 3.99
mid_peak_rate = 11.76
std_rate = 14.23

def get_tou_tariff(date):
    # https://www.londonhydro.com/accounts-services/electricity-rates

    month = date.month
    hour = date.hour

    if month >= 5 and month <= 10:
        #peak
        if (hour >= 11 and hour <= 17):
            return  peak_rate
        # off peak
        elif (hour > 17 and hour < 19) or (hour > 5 and hour < 11):
            return mid_peak_rate
        else: 
            return off_peak_rate
    else:
        #peak
        if (hour >= 7 and hour <= 11) or (hour >= 17 and hour <= 19):
            return peak_rate
        #off peak
        elif hour > 11 and hour < 17:
            return mid_peak_rate
        #mid peak
        else: 
            return  off_peak_rate
        
merged_df["rate"] = np.nan
merged_df.loc[merged_df[merged_df["stdorToU"] == "Std"].index,"rate"] = std_rate
merged_df.loc[merged_df["rate"].isna(),"rate"] = merged_df.loc[merged_df["rate"].isna(), "datetime"].apply(lambda date: get_tou_tariff(date))
merged_df["cost"] = merged_df["rate"] * merged_df["kWh"] / 100

# assign season
season_mask = (merged_df["datetime"].dt.month >= 5) & (merged_df["datetime"].dt.month <= 10)
merged_df["season"] = "winter"
merged_df.loc[season_mask, "season"] = "summer"


In [ ]:
# Tou rates

tou_rate_grouped = merged_df[merged_df["stdorToU"] == "ToU"].pivot_table(index="time", columns="season", values="rate", aggfunc=np.median, sort=False)
chart = sns.lineplot(data=tou_rate_grouped)
chart.set_xlabel("Time of Day")
chart.set_ylabel("pence/kWh")
chart.set_title("Time of Use Rates by Season")
chart.legend(loc="upper left", bbox_to_anchor=(1,1))
chart.set_xticks(np.arange(4,48,8));


In [ ]:
pivoted_usage = merged_df.pivot_table(index=["month", "date"], columns="Acorn_grouped", values="kWh", aggfunc=np.sum, sort=False).reset_index()
median_monthly_usage  = pivoted_usage.drop(columns=["date"]).groupby("month").median()
chart = sns.lineplot(data=median_monthly_usage)
chart.set_xlabel("Month")
chart.set_ylabel("kWh Usage")
chart.set_title("Monthly kWh Usage")
chart.legend(loc="upper left", bbox_to_anchor=(1,1), title="Acorn Group")
chart.set_xticks(np.arange(13));

In [ ]:
# Data stats

display(merged_df.drop_duplicates(subset=["LCLid"]).groupby(["Acorn", "Acorn_grouped", "stdorToU"]).count()["LCLid"])

In [ ]:
# Seasonal energy usage by tariff

seasonal_energy_usage = merged_df.pivot_table(index="time", columns=["stdorToU", "season"], values="kWh", aggfunc=np.median, sort=False)
seasonal_energy_usage.columns = seasonal_energy_usage.columns.to_flat_index()
column_names = []
for column in seasonal_energy_usage.columns:
    column_names.append("-".join(list(column)))

seasonal_energy_usage.columns = column_names
chart = sns.lineplot(data=seasonal_energy_usage)
chart.set_xlabel("Time of Day")
chart.set_xlabel("Time of Day")
chart.set_ylabel("kWh Usage")
chart.set_title("kWh Usage by Season and Tariff, All groups")
plt.legend(loc="upper left", bbox_to_anchor=(1,1))
chart.set_xticks(np.arange(4,48,8));


In [ ]:
# Separate each Acorn group to see variation

acorn_groups = merged_df["Acorn"].unique().tolist()
acorn_groups_dict = {}
for group in acorn_groups:
    acorn_groups_dict[group] = merged_df[merged_df["Acorn"] == group].reset_index(drop=True)

fig, axes = plt.subplots(len(acorn_groups), sharex=True, figsize=(6, 12))
for group, df in acorn_groups_dict.items():

    group_df = df.pivot_table(index="time", columns=["stdorToU", "season"], values="kWh", aggfunc=np.median, sort=False)
    group_df.columns = [f"{col[0]}-{col[1]}" for col in group_df.columns.to_flat_index()]
    axis = axes[acorn_groups.index(group)]
    sns.lineplot(data=group_df, ax=axis)
    axis.set_ylabel("kWh Usage")
    axis.set_title(group)
    axis.legend(loc="upper left", bbox_to_anchor=(1,1))

# Set x-axis ticks for all subplots
for axis in axes:
    axis.set_xticks(np.arange(4, 48, 8))

plt.suptitle("kWh Usage by Season and Tariff by Acorn group");

In [ ]:
# Monthly bill ToU vs Std rate

total_energy_cost = merged_df.pivot_table(index=["month", "LCLid"], columns = ["stdorToU"], values = "cost", aggfunc=np.sum).reset_index()
per_house_energy_cost = total_energy_cost.drop(columns=["LCLid"]).groupby("month").median()[["Std", "ToU"]]
per_house_energy_cost = per_house_energy_cost.stack().reset_index()
chart = sns.barplot(x=per_house_energy_cost["month"], y=per_house_energy_cost[0], hue=per_house_energy_cost["stdorToU"])
chart.set_xlabel("Months")
chart.set_ylabel("Monthly Bill, Pounds")
chart.set_title("Median Monthly Bill by Tariff")
plt.legend(title="Tariff",loc="upper left", bbox_to_anchor=(1,1));


In [ ]:
# Peak Load Analysis Total 
load_grouped = merged_df.pivot_table(index = "date", columns="time", values="kWh", aggfunc=np.sum, sort=False).dropna()
load_max = load_grouped.max()
load_median = load_grouped.median()
load_profile = pd.concat([load_median, load_max], axis=1)
load_profile.columns = ["median kW", "peak kW"]
chart = sns.lineplot(data=load_profile)
chart.set_xlabel("Time of Day")
chart.set_ylabel("kW Load")
chart.set_title("Load profile, Annual")
chart.legend(loc="upper left", bbox_to_anchor=(1,1))
chart.set_xticks(np.arange(4, 48, 8));

In [ ]:
# Peak Load Analysis Season 
load_grouped_season = merged_df.pivot_table(index="date", columns=["season", "time"], values="kWh", aggfunc=np.sum, sort=False)
load_max_winter = load_grouped_season["winter"].max()
load_max_summer = load_grouped_season["summer"].max()
load_profile_season = pd.concat([load_max_summer, load_max_winter], axis=1)
load_profile_season.columns=["summer peak kW", "winter peak kW"]
chart = sns.lineplot(data=load_profile_season)
chart.set_xlabel("Time of Day")
chart.set_ylabel("kW Load")
chart.set_title("Load profile, Seasonal")
chart.legend(loc="upper left", bbox_to_anchor=(1,1))
chart.set_xticks(np.arange(3, 48, 8));